In [1]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [ ]:
import requests,json
def get_current_weather(city:str,unit='kelvin')->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [3]:
get_current_weather("gurugram")

{'City Name': 'gurugram',
 'weather': 'haze',
 'temperature': 294.17,
 'unit': 'kelvin'}

# Tool Integration with LLM

- To integrate, we need to define tool metadata, it consists of 3 components
    - Name: A unique identifier for the tool.
    - Description: A brief explanation of what the tool does.
    - Parameters: The input parameters required by the tool, defined in JSON schema format.

In [4]:
tool_metadata = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to fetch current weather information for any city or location",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string",
                                  "description":"name of location, city e.g. mumbai, new york"}},
        },
        "required":["city"],
    }
},
]

In [10]:
response = client.chat.completions.create(model='gpt-4o-mini',
                               messages=[{"role":"user",'content':'what is the weather in mumbai today?'}],
                               tools=tool_metadata,tool_choice='auto')
print(response.to_json(indent=2))

{
  "id": "chatcmpl-CdVV0pHPnEMjjJ3FNLX54PHOkwSz8",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "tool_calls": [
          {
            "id": "call_Y8Y8E2Myox3jB7C0ftjLVNnI",
            "function": {
              "arguments": "{\"city\":\"mumbai\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1763531910,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_efad92c60b",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 78,
    "total_tokens": 95,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0


In [ ]:
toolmap = {"get_current_weather":get_current_weather}

import json
def generate_reponse(prompt):
    messages=[{"role":"system",'content':'you are helpful assistant'},
              {"role":"user",'content':prompt}]
    first_response = client.chat.completions.create(model='gpt-4o-mini',
                                                               messages=messages,
                               tools=tool_metadata,tool_choice='auto')
    
    if first_response.choices[0].message.tool_calls:
        tool_calls = first_response.choices[0].message.tool_calls
        print("LLM selected tool call: \n", tool_calls)
        messages.append(first_response.choices[0].message)

        for tool in tool_calls:
            tool_name = tool.function.name
            tool_obj = toolmap[tool_name]
            tool_args = tool.function.arguments
            tool_args = json.loads(tool_args)
            tool_response = tool_obj(**tool_args)
            messages.append({"tool_call_id":tool.id,"role":"tool",
                             'name':tool_name,"content":json.dumps(tool_response)})
        second_response = client.chat.completions.create(model="gpt-4o-mini",
                                                         messages=messages)
        
        return second_response.choices[0].message.content        
        # do something
    else:
        return first_response.choices[0].message.content


In [16]:
generate_reponse("what is artificial Intelligence?")

'Artificial Intelligence (AI) refers to the simulation of human intelligence processes by computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types:\n\n1. **Narrow AI**: This type of AI is designed and trained for a specific task. Examples include voice assistants like Siri or Alexa, recommendation algorithms used by streaming services, and image recognition software.\n\n2. **General AI**: Also known as Strong AI, this type of AI would have the ability to understand, learn, and apply intelligence in a way that is indistinguishable from human capabilities. This level of AI does not currently exist.\n\nAI encompasses various subfields, including machine learning (where computers learn from data), natural language processing (enabling machines to understand and respond to human language), robotics, and comput

In [17]:
generate_reponse("what is weather today in delhi?")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_zhN8l9NSNxiJEasjeRknTz3R', function=Function(arguments='{"city":"delhi"}', name='get_current_weather'), type='function')]


'Today in Delhi, the weather is clear sky with a temperature of approximately 24.44°C (297.59 K).'

In [18]:
generate_reponse("what is weather today in delhi and bangalore?")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_UwNm6blDiBZrke2KDdw4jWfL', function=Function(arguments='{"city": "Delhi"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_E838r12ArhAVNpbbJRLvYTDC', function=Function(arguments='{"city": "Bangalore"}', name='get_current_weather'), type='function')]


'Today in Delhi, the weather is clear sky with a temperature of approximately 24.4°C (297.59 K).\n\nIn Bangalore, the weather is characterized by scattered clouds with a temperature of roughly 25.1°C (298.2 K).'

In [ ]:
!pip install wikipedia --quiet

In [21]:
import wikipedia


def get_wiki_summary(query:str):
    response = wikipedia.summary(query)
    return response

In [22]:
get_wiki_summary("history of delhi")

"Delhi has been an important political centre of India as the capital of several empires. The recorded history of Delhi begins with the 8th century Tomar Rajput dynasty. It is considered to be a city built, destroyed and rebuilt several times, as outsiders who successfully invaded the Indian subcontinent would ransack the existing capital city in Delhi, and those who came to conquer and stay would be so impressed by the city's strategic location as to make it their capital and rebuild it in their own way.\nFrom the Ancient to the medieval era, Delhi was ruled by the powerful Rajput dynasties such as the Tomaras, Chauhans, and Gautamas. The Delhi Sultanate is the name given for a series of five successive dynasties, which remained as a dominant power of Indian subcontinent with Delhi as their capital.\nDuring the sultanate period, the city became a center for culture. The Delhi Sultanate came to an end in 1526, when Babur defeated the forces of the last Lodi sultan, Ibrahim Lodi at the 

In [31]:
tool_metadata = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to fetch current weather information for any city or location",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string",
                                  "description":"name of location, city e.g. mumbai, new york"}},
        },
        "required":["city"],
    }
},
{
    "type":"function",
    "function":{
        "name":"get_wikipedia_summary",
        "description":"this function can be used to retrieve any information about places, people and historical events do NOT USE IT FOR ANY OTHER USE CASE OTHER THATN PLACES PEOPLE AND HISTORICAL EVENTS.",
        "parameters":{
            "type":"object",
            "properties":{"query":{"type":"string",
                                  "description":"name of people, places or historical event e.g. history of delhi"}},
        },
        "required":["query"],
    }
},
]

In [32]:
toolmap = {"get_current_weather":get_current_weather,
           "get_wikipedia_summary":get_wiki_summary}

import json
def generate_reponse(prompt):
    messages=[{"role":"system",'content':'you are helpful assistant YOU MAKE USE OF TOOLS ONLY WHEN RELEVANT OR NECESSARY'},
              {"role":"user",'content':prompt}]
    while True:
        response = client.chat.completions.create(model='gpt-4o-mini', messages=messages,
                                tools=tool_metadata,tool_choice='auto')
        if response.choices[0].message.tool_calls:
            tool_calls = response.choices[0].message.tool_calls
            print("LLM selected tool call: \n", tool_calls)
            messages.append(response.choices[0].message)
            for tool in tool_calls:
                tool_name = tool.function.name
                tool_obj = toolmap[tool_name]
                tool_args = tool.function.arguments
                tool_args = json.loads(tool_args)
                tool_response = tool_obj(**tool_args)
                messages.append({"tool_call_id":tool.id,"role":"tool",
                                'name':tool_name,"content":json.dumps(tool_response)})
        else:
            break
    return response.choices[0].message.content


In [33]:
generate_reponse("what is AI?")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_bVWLQVdKLsfzWM44o7tlj4EK', function=Function(arguments='{"query":"Artificial Intelligence"}', name='get_wikipedia_summary'), type='function')]


"Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use intelligence to take actions that maximize their chances of achieving defined goals.\n\nHigh-profile applications of AI include:\n\n- Advanced web search engines (e.g., Google Search)\n- Recommendation systems (used by YouTube, Amazon, and Netflix)\n- Virtual assistants (e.g., Google Assistant, Siri, and Alexa)\n- Autonomous vehicles (e.g., Waymo)\n- Generative tools (e.g., language models and AI art)\n\nVarious subfields of AI research focus on particular goals and tools such as learning, reasoning, knowledge representation, natural language processing, and perception.\n\nAI has a rich history, having been founded as an acad

In [34]:
generate_reponse("what is weather today in delhi")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_ZpYRcKe3KPch5LXXk52njild', function=Function(arguments='{"city":"Delhi"}', name='get_current_weather'), type='function')]


'Today in Delhi, the weather is clear sky with a temperature of approximately 298.66 K (which is about 25.5 °C or 77.9 °F).'

In [35]:
generate_reponse("tell me more about delhi city")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_PjYBNO18CTrRdeMaMQMr8lAH', function=Function(arguments='{"query":"Delhi"}', name='get_wikipedia_summary'), type='function')]
LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_n2aJ45eKBD9tVVQzK1Lf1cCf', function=Function(arguments='{"query":"Delhi city"}', name='get_wikipedia_summary'), type='function')]


"Delhi, officially known as the National Capital Territory (NCT) of Delhi, is a city and union territory in India that includes New Delhi, the country's capital. Located along the banks of the Yamuna River, Delhi shares borders with Uttar Pradesh to the east and Haryana to the west, north, and south.\n\n### Key Facts About Delhi:\n- **Area**: The NCT of Delhi covers approximately 1,484 square kilometers (573 square miles).\n- **Population**: According to the 2011 census, the population of Delhi's city proper was over 11 million, while the NCT's population was about 16.8 million. Today, the urban agglomeration, which includes satellite cities like Ghaziabad, Faridabad, Gurgaon, and Noida, has an estimated population exceeding 28 million.\n- **Historical Significance**: \n  - Delhi has a rich history, having served as the capital for two major empires: the Delhi Sultanate and the Mughal Empire. \n  - It is home to three UNESCO World Heritage Sites: the Qutub Minar, Humayun's Tomb, and th

In [36]:
generate_reponse("what is weather today in delhi and mumbai")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_zutGRYf7oYJZMQWRdWKzR5du', function=Function(arguments='{"city": "Delhi"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_hcXKoMqDYXPNNNBICDHBXHPq', function=Function(arguments='{"city": "Mumbai"}', name='get_current_weather'), type='function')]


'Today in Delhi, the weather is clear sky with a temperature of approximately 25.5°C (298.66 K). \n\nIn Mumbai, the weather is smoky with a temperature of approximately 30.0°C (303.14 K).'

In [37]:
generate_reponse("tell me about city delhi and the weather information there.")

LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_3vUyDsF6ChCkmHfMbY7AY8El', function=Function(arguments='{"query": "Delhi"}', name='get_wikipedia_summary'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_FmvRhpZ8iaVOvPZmM07lFEgL', function=Function(arguments='{"city": "Delhi"}', name='get_current_weather'), type='function')]


"### Overview of Delhi\nDelhi, officially known as the National Capital Territory of Delhi (NCT), is the capital city of India. It serves as a major center for politics, culture, and history. The city is divided into two parts: Old Delhi, which has historical significance, and New Delhi, which serves as the seat of government. Delhi has a diverse population, rich cultural heritage, and numerous landmarks like the India Gate, Red Fort, and Qutub Minar. It also plays a crucial role in India's economy and education sector.\n\n### Current Weather in Delhi\n- **Weather Condition:** Clear sky\n- **Temperature:** Approximately 298.66 K (about 25.5 °C or 77.9 °F)\n\nLet me know if you need more information!"

In [40]:
generate_reponse("fetch the name of city where the first mughal war occured and tell me the weather information of that city today.")


LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_hsDpqJWJBQ6WdVICoMGmfjXS', function=Function(arguments='{"query":"first mughal war"}', name='get_wikipedia_summary'), type='function')]
LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_MJ6rcdvihPdqX8EODsxvh9aL', function=Function(arguments='{"query":"first mughal war location"}', name='get_wikipedia_summary'), type='function')]
LLM selected tool call: 
 [ChatCompletionMessageFunctionToolCall(id='call_Hyr4rK0ETWr6nFA6N1mcRGmM', function=Function(arguments='{"city":"Mumbai"}', name='get_current_weather'), type='function')]


"The first Mughal war took place near the city of Mumbai (formerly Bombay).\n\nToday's weather in Mumbai:\n- **Condition:** Smoke\n- **Temperature:** 303.14 K (approximately 30 °C)"

In [41]:
generate_reponse('what is current time in UK?')


"The current time in the UK can vary depending on whether it is observing Daylight Saving Time (BST) or Standard Time (GMT). The UK is generally in the GMT time zone but switches to BST during the warmer months.\n\nYou can easily check the current time through various websites or devices that display time. If you need to know the exact current time, please use your device's time settings. "